In [91]:
#!conda install -c conda-forge beautifulsoup4
from bs4 import BeautifulSoup
#!conda install -c conda-forge lxml
import requests
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium
import folium
import pandas as pd
from pandas.io.json import json_normalize

### 1. Scrape the Wikipedia page to get data

In [92]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
#print(soup.prettify())

In [93]:
table=soup.find('table',class_='wikitable sortable')
data=table.find_all('td')
Postcode=[]
Borough=[]
Neighborhood=[]
for item in data[0::3]:
    Postcode.append(item.text)
for item in data[1::3]:
    try:
        Borough.append(item.a.text)
    except:
        Borough.append(None)
for item in data[2::3]:
    try:
        Neighborhood.append(item.a.text)
    except:
        Neighborhood.append(None)

### 2. Create the dataframe

In [94]:
df_data={'Postcode':Postcode,'Borough':Borough,'Neighborhood':Neighborhood}
df=pd.DataFrame(df_data,columns=['Postcode','Borough','Neighborhood'])
df.shape

(288, 3)

#### Remove cells with a borough that is Not assigned

In [95]:
df.dropna(subset=['Borough'],inplace=True)
df.reset_index(drop=True,inplace=True)
df.head()

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


#### Assign neighborhoods that are Not assigned to their borough

In [96]:
missing_index=df[df['Neighborhood'].isna()].index
df.loc[missing_index,'Neighborhood']=df.loc[missing_index,'Borough']
df.head(12)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


#### Combine multiple neighborhoods with the same postcode togeter, separated with ',' 

In [102]:
df.drop_duplicates(inplace=True)
Toronto_Neighborhoods=df.groupby(['Postcode','Borough'])['Neighborhood'].apply(lambda x: ', '.join(x)).to_frame().reset_index()
Toronto_Neighborhoods.head(20)

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Scarborough, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Scarborough
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Scarborough, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough"
9,M1N,Scarborough,"Birch Cliff, Scarborough"


In [103]:
Toronto_Neighborhoods.shape

(100, 3)